<a href="https://colab.research.google.com/github/manan-arya/ChatBot/blob/master/Sentiment_Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing libraries
import nltk
from nltk import FreqDist
import pandas as pd
import numpy as np
import re
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import seaborn as sns
from nltk.corpus import wordnet

In [68]:
!wget https://raw.githubusercontent.com/manan-arya/ChatBot/master/reviews2.csv

--2019-01-28 13:39:47--  https://raw.githubusercontent.com/manan-arya/ChatBot/master/reviews2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2249109 (2.1M) [text/plain]
Saving to: ‘reviews2.csv.3’

reviews2.csv.3      100%[===================>]   2.14M  --.-KB/s    in 0.07s   

2019-01-28 13:39:48 (29.2 MB/s) - ‘reviews2.csv.3’ saved [2249109/2249109]



In [0]:
# findinf frequence of words
def freq_words(x, terms = 30):
    all_words = ' '.join([str(text) for text in x])
    all_words = all_words.split()
  
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
    
    # selecting top 20 most frequent words
    d = words_df.nlargest(columns="count", n = terms) 
    plt.figure(figsize=(20,5))
    ax = sns.barplot(data=d, x= "word", y = "count")
    ax.set(ylabel = 'Count')
    plt.show()


In [0]:
# function to remove stopwords
def remove_stopwords(rev):
  synonyms = [] 
  antonyms = [] 
  
  for syn in wordnet.synsets("easy"): 
      for l in syn.lemmas(): 
          synonyms.append(l.name()) 
          if l.antonyms(): 
             antonyms.append(l.antonyms()[0].name()) 
  
  #print(set(synonyms)) 
  #print(set(antonyms))
  
  stop_words = stopwords.words('english') + synonyms + antonyms
  rev_new = " ".join([i for i in rev if i not in stop_words])
  return rev_new

In [0]:
# user porterstemmer for stemming
def stemming(texts): 
    s_stemmer = PorterStemmer()
    for i in range(len(texts)):
        texts[i]=[s_stemmer.stem(texts[i][j]) for j in range(len(texts[i]))]
        return texts

In [0]:
# Read the csv file
df = pd.read_csv('reviews2.csv')


# replace "n't" with " not"
df['Review'] = df['Review'].str.replace("n\'t", " not")

# remove unwanted characters, numbers and symbols
df['Review'] = df['Review'].str.replace("[^a-zA-Z#]", " ")

# remove short words (length < 3)
df['Review'] = df['Review'].apply(lambda x: ' '.join([str(w) for w in str(x).split() if len(w)>2]))

In [0]:
# remove stopwords from the text
reviews = [remove_stopwords(r.split()) for r in df['Review']]

# make entire text lowercase
reviews = [r.lower() for r in reviews]
        
tokenized_reviews = pd.Series(reviews).apply(lambda x: x.split())
#reviews_2 = stemming(tokenized_reviews)

In [88]:
tokenized_reviews

0                                      [good, interesting]
1        [class, helpful, currently, still, learning, c...
2        [easy, follow, includes, lot, basic, important...
3                   [structured, approach, thank, sharing]
4        [great, piece, work, especially, liked, lifeha...
5        [excellent, representation, day, day, thanks, ...
6                        [interesting, welldesigned, mooc]
7                                          [good, quality]
8                                                 [thanks]
9        [excellent, overview, ways, printing, use, may...
10       [opened, understanding, could, accomplished, p...
11       [important, information, copy, rights, patents...
12       [really, good, survey, class, potential, print...
13                                           [interesting]
14                           [full, individuals, speaking]
15       [less, repetitive, full, useful, informations,...
16                         [good, inspirational, example

In [75]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [104]:
tagged = []
for i in range(len(tokenized_reviews)):
  tagged.append(nltk.pos_tag(tokenized_reviews[i]))
tagged
#namedEnt = nltk.ne_chunk(tagged, binary=True)
#namedEnt

[[('good', 'JJ'), ('interesting', 'NN')],
 [('class', 'NN'),
  ('helpful', 'NN'),
  ('currently', 'RB'),
  ('still', 'RB'),
  ('learning', 'VBG'),
  ('class', 'NN'),
  ('makes', 'VBZ'),
  ('lot', 'NN'),
  ('basic', 'JJ'),
  ('music', 'NN'),
  ('knowledge', 'NN')],
 [('easy', 'JJ'),
  ('follow', 'JJ'),
  ('includes', 'VBZ'),
  ('lot', 'NN'),
  ('basic', 'JJ'),
  ('important', 'JJ'),
  ('techniques', 'NNS'),
  ('use', 'VBP'),
  ('sketchup', 'NN')],
 [('structured', 'VBN'),
  ('approach', 'NN'),
  ('thank', 'NN'),
  ('sharing', 'VBG')],
 [('great', 'JJ'),
  ('piece', 'NN'),
  ('work', 'NN'),
  ('especially', 'RB'),
  ('liked', 'JJ'),
  ('lifehacks', 'NNS'),
  ('cio', 'VBP')],
 [('excellent', 'JJ'),
  ('representation', 'NN'),
  ('day', 'NN'),
  ('day', 'NN'),
  ('thanks', 'NNS'),
  ('sharing', 'VBG'),
  ('vision', 'NN'),
  ('experience', 'NN'),
  ('advicesregardsdaniel', 'NN'),
  ('damas', 'NN')],
 [('interesting', 'VBG'), ('welldesigned', 'VBD'), ('mooc', 'NNS')],
 [('good', 'JJ'), ('qua

In [108]:
reviews_2 = []
noun_list = ['NN','NNS','NNP','NNPS']
for i in range(len(tagged)):
  for j in range(len(tagged[i])):
    if tagged[i][j][1] in noun_list:
      reviews_2.append(tagged[i][j][0])
print(reviews_2)

['interesting', 'class', 'helpful', 'class', 'lot', 'music', 'knowledge', 'lot', 'techniques', 'sketchup', 'approach', 'thank', 'piece', 'work', 'lifehacks', 'representation', 'day', 'day', 'thanks', 'vision', 'experience', 'advicesregardsdaniel', 'damas', 'mooc', 'quality', 'thanks', 'overview', 'ways', 'use', 'future', 'understanding', 'printing', 'information', 'copy', 'rights', 'patents', 'survey', 'class', 'printing', 'examples', 'individuals', 'others', 'obstacles', 'curve', 'investment', 'ways', 'solutions', 'problems', 'thing', 'challengesthanks', 'individuals', 'informations', 'applications', 'inspirational', 'examples', 'introduction', 'applications', 'printing', 'overview', 'technology', 'examples', 'world', 'problems', 'indepth', 'applications', 'sections', 'development', 'education', 'class', 'ramifications', 'change', 'economy', 'review', 'printing', 'class', 'lot', 'sense', 'skip', 'cost', 'introduction', 'printing', 'situation', 'printing', 'technology', 'develop', 'bus

In [109]:
# Create the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(reviews_2)#reviews_2)

# Convert list of reviews (reviews_2) into a Document Term Matrix using the dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]#reviews_2]
   

TypeError: ignored

In [78]:
reviews# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix,
                id2word=dictionary,
                num_topics=7, 
                random_state=100,
                chunksize=1000,
                passes=50)

lda_model.print_topics()

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

# Print the Keyword in the 10 topics
lda_model.print_topics() 

NameError: ignored

In [0]:
# Print the Keyword in the 10 topics
lda_model.print_topics() 